# Experimento
## Comparação entre algoritmos de busca/otimização de hiper-parâmetros


### Motivação


Como o espaço de busca dos parâmetros para cada técnica de Machine Learning costuma ser contínuo e de alta dimensionalidade, para atingir os valores ótimos é necessário uma busca exaustiva de baixa granularidade, demandando assim um alto custo computacional, para resolver esse problema existem técnicas que servem para realizar essa busca de maneira a diminuir os custos de tempo e processamento.


### Objetivo

Este experimento tem como objetivo verificar quais das funções disponíveis atingem o melhor resultado dentre as séries de retorno das ações componentes do índice IBOVESPA.

As técnicas a serem comparadas serão:
* Random Search. [1]
* Grid Search. [2]
* Particle Swarm Optimization (PSO). [3]
* Nelder-Mead. [4]
* Covariance Matrix Adaptation Evolution Strategy (CMA-ES). [5]

### Metodologia

1. Primeiro passo é conseguir a base de dados, foi possível através do sistema do Yahoo! Finance [6] conseguir alguns dos dados do IBOVESPA ( 43 dos 70 componentes ).

2. Transformar a série de preços em valores de retorno e criar uma matriz de padrões de entrada e saída, segundo o parâmetro de número de lags, para esse experimento foi escolhido o valor 4.

3. Para conseguir alguma relevância estatística serão feitas 50 execuções de um cross-validation [7] para cada algoritmo, a variável a ser analisada será o valor da raiz quadrada do erro médio quadrático (RMSE) de cada iteração.

4. Será gerada então uma matriz de 50 linhas e 5 colunas contendo todos os resultados obtidos, a primeira análise feita é se a distribuição dos valores de RMSE de cada técnica é igual, para tal foi usado um teste estatístico não-paramétrico, o **'Kruskal-Wallis H-test'**. Esse teste é o equivalente não-paramétrico do ANOVA e sua hipótese nula afirma que as distribuições dos grupos são iguais, se rejeitada é calculada uma matriz de 5 linhas e 5 colunas para verificar essas diferenças, cada elemento da matriz é o resultado de um teste não-paramétrico entre os pares de técnicas 'i' e 'j', o teste é o mesmo realizado anteriormente, porém quando o número de grupos é 2 (teste pareado) ele é conhecido por **'Mann–Whitney U test'**, equivalente não paramétrico do t-test.

5. O mesmo procedimento é feito para os tempos de execução.

6. Se as matrizes forem criadas, ou seja, existe diferença estatística entre os algoritmos de busca, é calculado um vetor com a soma de cada linha da matriz para criar um ranking dessa técnica em relação as outras. 

Por exemplo, temos uma matriz formada por 0's e 1's, onde uma linha representa o desempenho de uma técnica em relação as outras, se for 0 temos a possibilidade de que as técnicas não apresentam diferença estatística entre si ou então o i-ésimo algoritmo é melhor que o j-ésimo ( apresenta um valor médio menor, nesse caso melhor pois tratamos de erro ), se for 1 só existe a possibilidade que existe diferença estatística entre as técnicas e que a i-ésima apresenta um pior desempenho ( valor médio maior ).

Então cada elemento do vetor varia entre 0 e 4, onde o primeiro representa que aquele algoritmo não foi pior que nenhum outro e 4 indica que ele foi pior que todos os outros.

Esse experimento foi feito utilizando 3 diferentes regressores, para verificar se os resultados eram consistentes entre diferentes técnicas de Machine Learning, foram escolhidos o **elmK**, **elmR** e **svr**.

### Descrição

### Código - Inicialização

Importação da biblioteca **pai** ( Portfolio-AI ) e definição dos algoritmos de busca.

In [3]:
from tools import *

import pai
from pai.portfolio import get_data


assets = open('ibovespa.txt', 'r').read().split('\n')
results_path = "results/"
search_functions = ["random search", "grid search", "particle swarm", "nelder-mead", "cma-es"]


### Código - Run

Para realizar a busca através do método regressor.search_param() foram utilizados alguns parâmetros extras:
* Tipo do cross-validation: time series cross-validation
* Número de folds do cross-validation: 10
* Número de iterações do algoritmo de busca: 50
* Função objetivo a ser otimizada: RMSE


In [5]:
def run(name):

    # Init regressor
    r = pai.Regressor(name)

    if not os.path.exists(results_path):
        os.makedirs(results_path)

    # Iterate over assets list
    for i, asset in enumerate(assets):

        filename = results_path + name + "_" + asset + ".p"

        # Check if need to run this asset
        if check(filename):
            try:
                # Get data and create dataset matrix
                stock = get_data(asset)
                stock.create_database(4, series_type="return")
                data = stock.get_database()

                result = {"finished": False}

                for f in search_functions:
                    result[f] = {}

                    print(asset, f)

                    metrics = []
                    time = []
                    for it in range(50):
                        start = datetime.datetime.now()
                        r.search_param(database=data, cv="ts", cv_nfolds=10,
                                       of="rmse", opt_f=f, eval=50,
                                       print_log=False,
                                       kf=["rbf"], f=["sigmoid"])
                        end = datetime.datetime.now()
                        delta = end - start

                        time.append(delta.total_seconds())
                        metrics.append(r.regressor.cv_best_error)

                    result[f]["cv"] = metrics
                    result[f]["cv_mean"] = np.mean(metrics)
                    result[f]["cv_std"] = np.std(metrics)

                    result[f]["time"] = time
                    result[f]["time_mean"] = np.mean(time)
                    result[f]["time_std"] = np.std(time)

                cvs = [result[f]["cv"] for f in search_functions]
                ts = [result[f]["time"] for f in search_functions]

                # Create paired test matrices only if exists differences among
                # algorithms distributions
                if not non_parametric_check_samples(cvs):
                    cv_nparam = matrix_non_parametric_paired_test(cvs)
                    result["cv_nparam"] = cv_nparam

                if not non_parametric_check_samples(ts):
                    t_nparam = matrix_non_parametric_paired_test(ts)
                    result["t_nparam"] = t_nparam

                result["finished"] = True

                # Save data
                pickle.dump(result, open(filename, "wb"))

            except:
                print("Error: ", asset)


### Código - Analysis

In [6]:
def analysis(name):
    time_means_mx = []
    cv_means_mx = []
    ranks_mx = []

    for asset in assets:
        filename = results_path + name + "_" + asset + ".p"

        try:
            result = pickle.load(open(filename, 'rb'))

            if result["finished"]:
                print(name, "Asset: ", asset)

                if "cv_nparam" not in result:
                    print()
                    for i in search_functions:
                        print(i)
                        print(result[i]["cv"])

                    tests = [result[sf]["cv"] for sf in search_functions]
                    print(non_parametric_check_samples(tests))
                    test_nparam = matrix_non_parametric_paired_test(tests)
                    print("cv_nparam")
                    print(test_nparam)

                ranks = []
                time_means = []
                cv_means = []
                for i in range(result["cv_nparam"].shape[0]):
                    cv_means.append(result[search_functions[i]]["cv_mean"])
                    time_means.append(result[search_functions[i]]["time_mean"])
                    ranks.append(np.sum(result["cv_nparam"][i, :]))

                    sum = np.sum(result["cv_nparam"][i, :])
                    print(search_functions[i], " is worst than ", sum, " functions")
                print()

                cv_means_mx.append(cv_means)
                time_means_mx.append(time_means)
                ranks_mx.append(ranks)

        except:
            pass
            # print("Error: ", asset)
            # print()

    cv = np.array(cv_means_mx)
    ts = np.array(time_means_mx)
    rk = np.array(ranks_mx)

    cv_mean, cv_std = mean_std(cv, "cv")
    ts_mean, ts_std = mean_std(ts, "ts")
    rk_mean, rk_std = mean_std(rk, "rk")

    print()
    print("Search Function | Ranking Sums")
    for j, sf in enumerate(search_functions):print(sf, ": ", np.sum(rk[:, j]))
    print()

    rk_samples = [rk[:, j] for j in range(rk.shape[1])]
    print("Rankings have same distribution: ",
          non_parametric_check_samples(rk_samples))

    rk_final = matrix_non_parametric_paired_test(rk_samples)
    # print("Rk final:")
    # print(rk_final)
    # print()

    print(rk.shape)
    print("(Search Function | Rank | Rank Mean | Time Mean)")
    rank = [(search_functions[i], np.sum(rk_final[i, :]), rk_mean[i], ts_mean[i]) for i in range(rk_final.shape[0])]
    rank = sorted(rank, key=lambda f: f[1] + f[2])
    print()
    for r in rank:print(r)


### Código - Run elmK

Rodando a análise dos dados gerados pelo regressor **elmK**.

In [7]:
analysis("elmk")

elmk Asset:  ABEV3.SA
random search  is worst than  2.0  functions
grid search  is worst than  3.0  functions
particle swarm  is worst than  0.0  functions
nelder-mead  is worst than  4.0  functions
cma-es  is worst than  0.0  functions

elmk Asset:  BBAS3.SA
random search  is worst than  3.0  functions
grid search  is worst than  0.0  functions
particle swarm  is worst than  2.0  functions
nelder-mead  is worst than  2.0  functions
cma-es  is worst than  1.0  functions

elmk Asset:  BBDC3.SA
random search  is worst than  3.0  functions
grid search  is worst than  0.0  functions
particle swarm  is worst than  1.0  functions
nelder-mead  is worst than  4.0  functions
cma-es  is worst than  1.0  functions

elmk Asset:  BBSE3.SA
random search  is worst than  2.0  functions
grid search  is worst than  1.0  functions
particle swarm  is worst than  1.0  functions
nelder-mead  is worst than  4.0  functions
cma-es  is worst than  0.0  functions

elmk Asset:  BISA3.SA
random search  is worst th

### Resultado - elmK

####1.

Search Function | Ranking Sums |
-----------------|--------------|
random search |  39.0 
grid search |  49.0 
particle swarm |  27.0 
nelder-mead |  163.0 
cma-es |  97.0 

O resultado do elmK mostra que o algoritmo que apresentou melhor desempenho foi o PSO, com o Random Search e Grid Search em seguida, respectivamente.

Além disso é feita mais uma checagem para dar uma força estatística ao resultado, é criada uma matriz contendo o vetor de rankings de cada uma das séries verificadas, formando assim uma matriz de 43 linhas e 5 colunas.

####2.

É feito um teste não-paramétrico para verificar se os rankings dos algoritmos apresentam uma mesma distribuição.

>Rankings have same distribution:  **False**

A hipótese nula foi negada, então é criada uma matriz quadrada contendo os resultados dos testes não-paramétricos pareados. A partir dela, gera-se um vetor final contendo o  ranking.

####3.

Com isso temos o resultado final:

Search Function | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
particle swarm | 0.0 | 0.62790697674418605| 1.6810304395348841
random search | 0.0 | 0.90697674418604646| 1.7933560576744183
grid search | 1.0 | 1.1395348837209303| 1.8438678865116276
cma-es| 3.0 | 2.2558139534883721| 1.6764245460465117
nelder-mead|  4.0| 3.7906976744186047| 0.19419816418604652

O PSO e Random Search apresentaram os melhores resultados, apesar de não existir uma diferença estatística entre os dois, o primeiro conseguiu uma menor média nos rankings e um menor tempo médio de execução.

### Resultado - elmR
####4.

Realizando a análise nos dados gerados pelo elmR:

Search Function | Ranking Sums |
-----------------|--------------|
random search |  76.0
grid search |  17.0
particle swarm |  40.0
nelder-mead |  172.0
cma-es |  63.0

>Rankings have same distribution:  **False**

Search Function | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
'grid search'  |  0.0  |  0.39534883720930231  |  3.8715486265116263)
'particle swarm'  |  1.0  |  0.93023255813953487  |  3.4902457665116291)
'cma-es'  |  2.0  |  1.4651162790697674  |  3.6026454469767444)
'random search'  |  2.0  |  1.7674418604651163  |  3.8385563106976743)
'nelder-mead'  |  4.0  |  4.0  |  0.4778094493023255)

### Resultado - svr
####5.

Realizando a análise nos dados gerados pelo svr:

Search Function | Ranking Sums |
-----------------|--------------|
random search |  18.0
grid search |  10.0
particle swarm |  30.0
nelder-mead |  128.0
cma-es |  89.0

>Rankings have same distribution:  **False**

Search Function | Rank | Rank Mean | Time Mean
------------------|----|----------|------------
'grid search'  |  0.0  |  0.29411764705882354  |  38.991934035294115
'random search'  |  0.0  |  0.52941176470588236  |  5.7148041870588235
'particle swarm'  |  1.0  |  0.88235294117647056  |  2.7880920964705886
'cma-es'  |  3.0  |  2.6176470588235294  |  0.29877295294117645
'nelder-mead'  |  4.0  |  3.7647058823529411  |  0.13051436647058823






### Conclusão

O PSO e Random Search ficaram empatados na primeira análise, não havendo diferença estatística significativa entre os 2, porém o PSO consegue ser o melhor numa taxa um pouco maior que o RS e num tempo menor.

Na segunda análise o PSO fica em segundo lugar, atrás do Grid Search, porém continua apresentando um tempo um pouco menor.

Esse resultado pode ser entendido pelo fato do elmR o espaço de busca ser menor, apenas 1 dimensão, do parâmetro de regularização *C*, pois o número de neurônios está fixo em 500. Isso deve ajudar um pouco o grid search, deixando a sua busca bem mais refinada do que o PSO. Mas apesar disso o PSO ainda conseguiu ser mais rápido.

Na terceira análise o PSO também fica em segundo lugar, atrás do Grid e Random Search, porém chega a ser 1 ordem de grandeza mais rápido. Isso deve ocorrer pois o **svr** precisa resolver um problema de otimização para conseguir seus vetores de suporte, assim o grid deve entrar em áreas onde os parâmetros pedidos são muito dificeis e de convergência demorada.

Com isso, proponho usar o PSO por ser um algoritmo mais robusto e atingir resultados bons de performance em tempo ótimo.

### Referências

[1]
[2]
[3]
[4]
[5]